In [1]:
# 根据valid_pair文件，将align文件左侧一列 按蛋白质类别 分成许多txt
file_name = "/home/wngys/lab/DeepFold/pair/valid_pair.txt"
# file_name = "pair_/d1b77a2.txt"
dir_name = "/home/wngys/lab/DeepFold/pair/pair_score/"
dic = {}
with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]
        score = lines.split('\t')[2].split('\n')[0]
        if id1 not in dic:
            dic[id1] = []
        dic[id1].append((id2, score))

for id in dic.keys():
    f_name = dir_name + id + ".txt"
    f_out = open(f_name, "w")
    for ele in dic[id]:
        f_out.write(ele[0])
        f_out.write('\t')
        f_out.write(ele[1])
        f_out.write("\n")
    f_out.close()

In [ ]:
# 验证一下Pair文件夹 文件数目是否等于align文件左侧蛋白质集合 元素数目
dir = "Pair"

files_n = os.listdir(dir)

print(len(files_n))

In [ ]:
# 由Pair文件 将第二列TM-score转换为 bool 表示 positive or negtive
dir_pair = "Pair"
out_dir = "Pair_bool/"
p = 0.9
i = 0
for file_n in os.listdir(dir_pair):
    # if i >= 5:
    #     break
    file_name = os.path.join(dir_pair, file_n)
    Max_score = -1
    with open(file_name, "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            # id2 = lines.split('\t')[0]
            score = lines.split('\t')[1].split("\n")[0]
            score = float(score)
            Max_score = max(score, Max_score)

    threshold = Max_score *p
    # print(file_n)
    # print(f"Max_score:{Max_score}")
    f_w = open(out_dir + file_n, "w")

    with open(file_name, "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            id2 = lines.split('\t')[0]
            score = lines.split('\t')[1].split("\n")[0]
            score = float(score)
            f_w.write(id2)
            f_w.write("\t")

            if score >= threshold:
                # print(id2, score)
                f_w.write(str(1))
                f_w.write("\n")
            else:
                f_w.write(str(0))
                f_w.write("\n")
    f_w.close()

    # i += 1
    # print("------------------------------------------------")